In [ ]:
import sys

class WaterJugsGame:
    def __init__(self, A, B, T):
        self.A = A  # Capacity of jug A
        self.B = B  # Capacity of jug B
        self.T = T  # Target volume
        self.goal_state = T  # Goal state
        self.visited = set()

    def is_goal(self, state):
        return state[0] == self.goal_state or state[1] == self.goal_state

    def get_successors(self, state):
        successors = []
        a, b = state
        # Fill jug A
        successors.append((self.A, b))
        # Fill jug B
        successors.append((a, self.B))
        # Empty jug A
        successors.append((0, b))
        # Empty jug B
        successors.append((a, 0))
        # Pour water from jug A to jug B
        pour = min(a, self.B - b)
        successors.append((a - pour, b + pour))
        # Pour water from jug B to jug A
        pour = min(b, self.A - a)
        successors.append((a + pour, b - pour))
        # Filter out already visited states
        return [s for s in successors if s not in self.visited]

    def minimax(self, state, depth, alpha, beta, maximizing_player):
        if self.is_goal(state) or depth == 0:
            return (1 if self.is_goal(state) else 0), state

        self.visited.add(state)

        if maximizing_player:
            max_eval = -sys.maxsize
            best_state = None
            for successor in self.get_successors(state):
                evaluation, _ = self.minimax(successor, depth - 1, alpha, beta, False)
                if evaluation > max_eval:
                    max_eval = evaluation
                    best_state = successor
                alpha = max(alpha, evaluation)
                if beta <= alpha:
                    break
            return max_eval, best_state
        else:
            min_eval = sys.maxsize
            best_state = None
            for successor in self.get_successors(state):
                evaluation, _ = self.minimax(successor, depth - 1, alpha, beta, True)
                if evaluation < min_eval:
                    min_eval = evaluation
                    best_state = successor
                beta = min(beta, evaluation)
                if beta <= alpha:
                    break
            return min_eval, best_state

    def play_game(self):
        current_state = (0, 0)
        player_turn = True  # True for user, False for AI

        while True:
            print(f"Current state: {current_state}")
            if self.is_goal(current_state):
                winner = "User" if player_turn else "AI"
                print(f"{winner} wins!")
                break

            if player_turn:
                print("User's turn")
                print("Select move:")
                print("1. Fill jug A")
                print("2. Fill jug B")
                print("3. Empty jug A")
                print("4. Empty jug B")
                print("5. Pour water from jug A to jug B")
                print("6. Pour water from jug B to jug A")
                move = int(input("Enter move number: "))

                if move == 1:
                    current_state = (self.A, current_state[1])
                elif move == 2:
                    current_state = (current_state[0], self.B)
                elif move == 3:
                    current_state = (0, current_state[1])
                elif move == 4:
                    current_state = (current_state[0], 0)
                elif move == 5:
                    pour = min(current_state[0], self.B - current_state[1])
                    current_state = (current_state[0] - pour, current_state[1] + pour)
                elif move == 6:
                    pour = min(current_state[1], self.A - current_state[0])
                    current_state = (current_state[0] + pour, current_state[1] - pour)
                else:
                    print("Invalid move")
                    continue
            else:
                print("AI's turn")
                _, current_state = self.minimax(current_state, 10, -sys.maxsize, sys.maxsize, True)

            player_turn = not player_turn

if __name__ == "__main__":
    A = int(input("Enter capacity of jug A: "))
    B = int(input("Enter capacity of jug B: "))
    T = int(input("Enter target volume T: "))
    game = WaterJugsGame(A, B, T)
    game.play_game()